In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

In [2]:
from liana.method.sp._Misty import MistyData
from liana.method.sp._misty_constructs import genericMistyData

In [3]:
adata = sc.read_h5ad('liana/tests/data/synthetic.h5ad')

In [4]:
misty = genericMistyData(intra=adata, set_diag=True)

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [5]:
misty

MuData object with n_obs × n_vars = 4205 × 33
  obs:	'cell_type'
  3 modalities
    intra:	4205 x 11
      obs:	'cell_type'
      obsm:	'spatial'
    juxta:	4205 x 11
      obsm:	'spatial'
      obsp:	'spatial_connectivities'
    para:	4205 x 11
      obsm:	'spatial'
      obsp:	'spatial_connectivities'

In [6]:
misty(n_estimators=20, bypass_intra=True)

In [7]:
misty.uns['target_metrics']

,target,intra_group,env_group,intra.R2,multi.R2,gain.R2,juxta,para
0,ECM,None,None,0,0.224915,0.224915,1.000000,0.000000
1,ligA,None,None,0,0.499275,0.499275,0.976263,0.023737
2,ligB,None,None,0,0.435992,0.435992,0.934720,0.065280
3,ligC,None,None,0,0.485529,0.485529,0.873449,0.126551
4,ligD,None,None,0,0.507503,0.507503,0.961657,0.038343
5,protE,None,None,0,0.035952,0.035952,1.000000,0.000000
6,protF,None,None,0,0.133950,0.133950,1.000000,0.000000
7,prodA,None,None,0,0.051568,0.051568,1.000000,0.000000
8,prodB,None,None,0,0.069874,0.069874,1.000000,0.000000
9,prodC,None,None,0,0.068745,0.068745,1.000000,0.000000


In [8]:
misty.uns['importances']

,target,predictor,intra_group,env_group,view,value
0,ECM,ligA,None,None,juxta,0.060344
1,ECM,ligB,None,None,juxta,0.075994
2,ECM,ligC,None,None,juxta,0.108937
3,ECM,ligD,None,None,juxta,0.067350
4,ECM,protE,None,None,juxta,0.233378
...,...,...,...,...,...,...
215,prodD,protE,None,None,para,0.103775
216,prodD,protF,None,None,para,0.139170
217,prodD,prodA,None,None,para,0.083315
218,prodD,prodB,None,None,para,0.108542


lrMistyData

In [9]:
from liana.testing._sample_anndata import generate_toy_spatial

In [10]:
from liana.resource import select_resource
from liana.method._pipe_utils import prep_check_adata, assert_covered
from liana.method._pipe_utils._pre import _get_props

from liana.method.sp._SpatialMethod import _SpatialMeta
from liana.method.sp._spatial_pipe import _get_ordered_matrix, _rename_means, \
    _run_scores_pipeline, _connectivity_to_weight, _handle_connectivity, _categorize
from liana.method.sp._bivariate_funs import _handle_functions

In [11]:
from liana.method.sp._lr_basis import _add_complexes_to_var

In [12]:
from liana.method.sp._misty_constructs import _make_view

In [13]:
from liana.method.sp._spatial_pipe import spatial_neighbors

In [14]:
adata = generate_toy_spatial()

In [15]:
use_raw = False
layer = None
verbose = True
resource = None
resource_name = 'consensus'
nz_threshold = 0.1

In [16]:
spatial_key = 'spatial'
bandwidth = 100
kernel = 'misty_rbf'
set_diag = True
cutoff = 0.1
zoi = 0

In [17]:
if resource is None:
    resource = select_resource(resource_name.lower())

In [18]:
views = dict()

In [19]:
adata

AnnData object with n_obs × n_vars = 700 × 765
    obs: 'bulk_labels', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'louvain'
    var: 'n_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'bulk_labels_colors', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'distances', 'connectivities', 'spatial_connectivities'

In [20]:
adata = prep_check_adata(adata=adata,
                         use_raw=use_raw,
                         layer=layer,
                         verbose=verbose,
                         groupby=None,
                         min_cells=None,
                         obsm = {spatial_key: adata.obsm[spatial_key]}
                         )

Using `.X`!
Converting mat to CSR format


In [21]:
adata = _add_complexes_to_var(adata,
                              np.union1d(resource['receptor'].astype(str),
                                         resource['ligand'].astype(str)
                                        )
                              )

In [22]:
adata

AnnData object with n_obs × n_vars = 700 × 769
    obs: 'bulk_labels', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'louvain'
    obsm: 'spatial'
    obsp: 'distances', 'connectivities', 'spatial_connectivities'

In [23]:
resource[resource['ligand'].str.contains('Penk', case=False)]

,ligand,receptor
1565,PENK,ADRA2A
1566,PENK,OPRM1
1567,PENK,OPRK1
1568,PENK,OPRL1
1569,PENK,MTNR1A
1570,PENK,GRM7
1571,PENK,MRGPRX1
1572,PENK,OGFR
3604,PENK,OPRD1


In [24]:
# filter_resource
resource = resource[(np.isin(resource.ligand, adata.var_names)) &
                    (np.isin(resource.receptor, adata.var_names))]

In [25]:
intra = _make_view(adata=adata[:, resource['receptor'].unique()],
                   nz_threshold=nz_threshold, obs=adata.obs)

views['intra'] = _add_complexes_to_var(intra, np.unique(resource['receptor']))

In [26]:
connectivity = spatial_neighbors(adata=adata, spatial_key=spatial_key,
                                 bandwidth=bandwidth, kernel=kernel,
                                 set_diag=set_diag, cutoff=cutoff,
                                 zoi=zoi, inplace=False)

extra = _make_view(adata=adata[:,resource['ligand'].unique()], spatial_key=spatial_key,
                   nz_threshold=nz_threshold, connecitivity=connectivity, obs=pd.DataFrame(index=adata.obs.index))
views['extra'] = _add_complexes_to_var(extra, np.unique(resource['ligand']))

In [27]:
from mudata import MuData

In [28]:
lr_misty = MistyData(data=views, obs=intra.obs, spatial_key=spatial_key, set_diag=True)

view intra is not a csr_matrix. Converting to csr_matrix
view extra is not a csr_matrix. Converting to csr_matrix


/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [32]:
lr_misty(bypass_intra=False)

In [30]:
lr_misty.uns['target_metrics'].sort_values('multi.R2')

,target,intra_group,env_group,intra.R2,multi.R2,gain.R2,extra
13,S1PR5,None,None,0,-0.177200,-0.177200,1.0
9,TNFRSF17,None,None,0,-0.146184,-0.146184,1.0
1,LAG3,None,None,0,-0.039979,-0.039979,1.0
7,TNFRSF13B,None,None,0,-0.031454,-0.031454,1.0
11,CD63,None,None,0,-0.020085,-0.020085,1.0
6,ITGA4,None,None,0,-0.017433,-0.017433,1.0
14,S1PR4,None,None,0,-0.015042,-0.015042,1.0
15,CD53,None,None,0,-0.013993,-0.013993,1.0
4,CD40,None,None,0,-0.007850,-0.007850,1.0
0,CD4,None,None,0,-0.007560,-0.007560,1.0


In [31]:
lr_misty.uns['importances'].sort_values('value')

,target,predictor,intra_group,env_group,view,value
249,TNFRSF17,S100A9,None,None,extra,0.006604
50,LAG3,C1QA,None,None,extra,0.007409
350,S1PR5,HLA-DRB5,None,None,extra,0.007918
232,HLA-DPB1,C1QB,None,None,extra,0.008606
338,S1PR5,HLA-DQA2,None,None,extra,0.008847
...,...,...,...,...,...,...
199,TNFRSF13B,MIF,None,None,extra,0.138574
259,CD74_CXCR4,HLA-DRA,None,None,extra,0.179820
250,TNFRSF17,MIF,None,None,extra,0.233979
357,S1PR5,SPON2,None,None,extra,0.293787
